In [ ]:
import nest_asyncio
import asyncio as asio

nest_asyncio.apply()

### Server side


In [ ]:
import nest_asyncio as asyncio
import os

import aiohttp.web

HOST = os.getenv('HOST', '127.0.0.1')
PORT = int(os.getenv('PORT', 8090))


async def testhandle(request):
    return aiohttp.web.Response(text='Test handle')


async def websocket_handler(request):
    print('Websocket connection starting')
    ws = aiohttp.web.WebSocketResponse()
    await ws.prepare(request)
    print('Websocket connection ready')

    async for msg in ws:
        print(msg)
        if msg.type == aiohttp.WSMsgType.TEXT:
            print(msg.data)
            if msg.data == 'close':
                await ws.close()
            else:
                await ws.send_str(msg.data + '/answer')

    print('Websocket connection closed')
    return ws


async def socket_server(testhandle, websocket_handler, HOST, PORT):
    app = aiohttp.web.Application()
    app.router.add_route('GET', '/', testhandle)
    app.router.add_route('GET', '/ws', websocket_handler)
    runner = aiohttp.web.AppRunner(app)
    await runner.setup()
    webhook_site = aiohttp.web.run_app(app, host=HOST, port=PORT)
    await webhook_site.start()

### Run server loop 

In [ ]:
loop_server = asio.get_event_loop()
loop_server.create_task(socket_server(testhandle, websocket_handler, HOST, PORT))

### Client 1 - background

In [ ]:
import aiohttp


URL = f'http://{HOST}:{PORT}/ws'


async def main():
    session = aiohttp.ClientSession()
    async with session.ws_connect(URL) as ws:

        await prompt_and_send(ws)
        async for msg in ws:
            print('Message received from server:', msg)
            await prompt_and_send(ws)

            if msg.type in (aiohttp.WSMsgType.CLOSED,
                            aiohttp.WSMsgType.ERROR):
                break


async def prompt_and_send(ws):
    new_msg_to_send = input('Type a message to send to the server: ')
    if new_msg_to_send == 'exit':
        print('Exiting!')
        raise SystemExit(0)
    await ws.send_str(new_msg_to_send)



loop_client = asio.get_event_loop()
loop_client.create_task(main())

### Client 2 - background

In [ ]:
# loop_client = asio.get_event_loop()
loop_client.create_task(main())

### Client 3 - Doing it

In [ ]:
## Run this until finished just for getting the output visible. Normally we don't want this to finish like above.

# loop_client = asio.get_event_loop()
loop_client.run_until_complete(main())